In [1]:
# importing os module for environment variables
import os
# importing necessary functions from dotenv library
from dotenv import load_dotenv 

import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

In [2]:
load_dotenv()


True

In [3]:
aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')
s3a_bucket_url = os.getenv('AWS_S3A_BUCKET_URL')

In [4]:
conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.11.1026") \
    .set("spark.hadoop.fs.s3a.access.key", aws_access_key_id) \
    .set("spark.hadoop.fs.s3a.secret.key", aws_secret_access_key)


In [5]:
#sc = SparkContext(conf=conf) -> error : multiple context
sc = SparkContext.getOrCreate(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set('fs.s3a.aws.credentials.provider','org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')

25/05/19 05:06:01 WARN Utils: Your hostname, Ayodeji-Lana resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/05/19 05:06:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/deljimae/spark/spark-3.5.5-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/deljimae/.ivy2/cache
The jars for the packages stored in: /home/deljimae/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0c12b714-bfc8-4f41-9670-03f2cabef2f6;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 197ms :: artifacts dl 7ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	:: evicted modules:
	com.amazonaws#aws-java-sdk-bundle;1.11.1026 by [com.amazonaws#aws-java-sdk-bundle;1.12.262] in [default]
	-----------------------------------------------------------

In [6]:
spark = SparkSession.builder \
    .config(conf=conf) \
    .getOrCreate()

In [9]:
df_green = spark.read.parquet(f'{s3a_bucket_url}/pq/green/*/*')

In [10]:
df_green.count()



2304517